# Pre-Processing

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 

# Create a spark session
spark = (
    SparkSession.builder.appName("preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "6g")
    .getOrCreate()
)

In [7]:
transactiondf1 = spark.read.parquet("../data/tables/transactions_20210228_20210827_snapshot/")
transactiondf2 = spark.read.parquet("../data/tables/transactions_20210828_20220227_snapshot/")
transactiondf3 = spark.read.parquet("../data/tables/transactions_20220228_20220828_snapshot/")
transactiondf12 = transactiondf1.union(transactiondf2)
transactiondf = transactiondf12.union(transactiondf3)
transactiondf.limit(5)

user_id,merchant_abn,dollar_value,order_id,order_datetime
18478,62191208634,63.255848959735246,949a63c8-29f7-4ab...,2021-08-20
2,15549624934,130.3505283105634,6a84c3cf-612a-457...,2021-08-20
18479,64403598239,120.15860593212783,b10dcc33-e53f-425...,2021-08-20
3,60956456424,136.6785200286976,0f09c5a5-784e-447...,2021-08-20
18479,94493496784,72.96316578355305,f6c78c1a-4600-4c5...,2021-08-20


In [8]:
userdf = spark.read.parquet("../data/tables/consumer_user_details.parquet")
userdf.limit(5)

user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975


In [9]:
consumerdf = spark.read.option("header","true").csv("../data/tables/tbl_consumer.csv", sep="|")
consumerdf = consumerdf.withColumnRenamed("name","customer_name")
consumerdf.limit(5)

customer_name,address,state,postcode,gender,consumer_id
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
Lindsay Jimenez,00653 Davenport C...,NSW,2780,Female,154128
Rebecca Blanchard,9271 Michael Mano...,WA,6355,Female,712975


In [10]:
merchantdf = spark.read.parquet("../data/tables/tbl_merchants.parquet")
merchantdf = merchantdf.withColumnRenamed("name","company_name")

# Replace all square brackets with round brackets
merchantdf = merchantdf.withColumn('tags', regexp_replace('tags', '\\[', '\\('))
merchantdf = merchantdf.withColumn('tags', regexp_replace('tags', '\\]', '\\)'))

# Extract take rate into seperate column
merchantdf = merchantdf.withColumn("take_rate", 
                                   split(col("tags"), "\\),").getItem(2))\
                       .withColumn('take_rate', 
                                   regexp_replace('take_rate', 'take rate: ', 
                                                  ''))\
                       .withColumn('take_rate', 
                                   regexp_replace('take_rate', '\\(', ''))\
                       .withColumn('take_rate', 
                                   regexp_replace('take_rate', '\\)', ''))

# Extract revenue band
merchantdf = merchantdf.withColumn("revenue_band", 
                                   split(col("tags"), "\\),").getItem(1))\
                       .withColumn('revenue_band', 
                                   regexp_replace('revenue_band', '\\(', ''))\
                       .withColumn('revenue_band', 
                                   regexp_replace('revenue_band', '\\)', ''))

# Extract tags band
merchantdf = merchantdf.withColumn("tags", 
                                   split(col("tags"), "\\),").getItem(0))\
                       .withColumn('tags', 
                                   regexp_replace('tags', '\\(', ''))\
                       .withColumn('tags', 
                                   regexp_replace('tags', '\\)', ''))\
                       .withColumn('tags', 
                                   regexp_replace('tags', ' +', ' '))\
                       .withColumn('tags', 
                                   lower('tags'))

merchantdf.limit(5)

company_name,tags,merchant_abn,take_rate,revenue_band
Felis Limited,"furniture, home f...",10023283211,0.18,e
Arcu Ac Orci Corp...,"cable, satellite,...",10142254217,4.22,b
Nunc Sed Company,"jewelry, watch, c...",10165489824,4.40,b
Ultricies Digniss...,"watch, clock, and...",10187291046,3.29,b
Enim Condimentum PC,music shops - mus...,10192359162,6.33,a


In [11]:
# Check no rows dropped when combining transactions with user
print(transactiondf.count(),userdf.count())
mergedf = transactiondf.join(userdf, "user_id")
print(mergedf.count())
print("\n")

# Check no rows dropped when combining with consumer
print(mergedf.count(), consumerdf.count())
mergedf = mergedf.join(consumerdf, "consumer_id")
print(mergedf.count())
print("\n")

# Check no rows dropped when combining with merchant
print(mergedf.count(),merchantdf.count())
mergedf = mergedf.join(merchantdf, "merchant_abn")
print(mergedf.count())

14195505 499999


14195505




14195505 499999


14195505




14195505 4026


13614675


We can see that number of rows goes down from 8151372 to 7817730. Since the join was on merchant_abn, this means that either the merchantdf didn't have those merchants on it or the merged df had incorrect merchant_abns

In [12]:
mergedf.write.parquet('../data/curated/mergedf.parquet')

AnalysisException: path file:/mnt/c/Users/Jai_b/Downloads/Uni/Year 3/Applied Data Science/generic-buy-now-pay-later-project-group-35/data/curated/mergedf.parquet already exists.

main websites out there containing australian datasets:
- https://explore.data.abs.gov.au/ 
- https://data.gov.au/
- https://researchdata.edu.au/
- https://aurin.org.au/

Key features to join on: 
- timestamp
- location: state/postcode/SA2 

Key model features: 
- merchant abn (nominal)
- datetime
- user id: how many users shop at each merchant
- dollar value
- location
- gender
- tags

SA2 Shapefile: https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files